In [3]:
import torch
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torchvision.transforms as transforms

from google.colab import drive
drive.mount('/content/drive')

# Dataset Class
class FashionMnist(Dataset):
    def __init__(self, data_path, is_train=True):
        filename = os.path.join(data_path, 'fashion-mnist_train.csv' if is_train else 'fashion-mnist_test.csv')
        assert os.path.exists(filename), 'File not found error'
        self.is_train = is_train
        self.data = pd.read_csv(filename)
        self.data = self.data.sort_values(by=['label']).to_numpy(dtype=np.float32)
        self.data_y = self.data[:, 0].astype(np.int)
        self.data_x = self.data[:, 1:]
        self.data_shape = (28, 28)
        self.transform = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.Grayscale(num_output_channels=1), # grayscale 변경
                transforms.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(75), 
                transforms.ToTensor()
            ]
        )
        self.transform_valid = transforms.Compose( ### test transform 추가
            [
                transforms.ToTensor()
            ]
        )

    def __len__(self):
        return len(self.data_y)

    def __getitem__(self, index):
        ret_x = np.reshape(self.data_x[index], self.data_shape)
        if self.is_train: # augmentation
            ret_x = self.transform(ret_x)
        else:
            ret_x = self.transform_valid(ret_x) ### 기존 ret_x는 np, 이는 GPU에서 사용 못함
                                                ### --> 텐서로 바꿔줌으로써 해결
        ret_y = self.data_y[index]
        return {
            'data_x': ret_x,
            'data_y': ret_y
        }

# Model Class
class ConvBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(ConvBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            # C -> M -> B -> R ==> C -> B -> R -> M
        )
  
    def forward(self, tensor):
        return self.block(tensor)

    
class BasicModel(nn.Module):
    def __init__(self, in_channel, num_classes):
        super(BasicModel, self).__init__()
        k = 32
        self.block = nn.Sequential(
            ConvBlock(in_channel, k),
            ConvBlock(k, k*2),
            ConvBlock(k*2, k*4),
            ConvBlock(k*4, k*2)
        )
        self.linear = nn.Sequential( # Linear --> Sequential
            nn.Linear(k*2, 100),
            nn.ReLU(),
            nn.Linear(100, num_classes)
        )
    
    def forward(self, tensor):
        out = self.block(tensor)
        out = out.view(-1, out.size(1))
        return self.linear(out)

def main():
    batch_size = 100 ### 3 -> 100
    epoch = 30
    learning_rate = 1e-03
    in_channel = 1 # grayscale --> not 3
    num_classes = 10
    betas = (0.5, 0.999)
    device = 'cuda' if torch.cuda.is_available() else 'cpu' 
    train_dataset = FashionMnist('/content/drive/My Drive', True)
    valid_dataset = FashionMnist('/content/drive/My Drive', False)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True, drop_last=False, pin_memory=True, num_workers=2)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle = True, drop_last=False, pin_memory=True, num_workers=2)
                                                                ### shuffle 추가

    model = BasicModel(in_channel=in_channel, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()# nn.BCELoss() # Binary Cross Entropy Loss ### 클래스가 여러개일 때 BCLE 사용 불가 (텐서 크기 안맞는 오류)
    optim = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=betas)
    for ep in range(epoch):
        # train
        avg_loss = 0
        avg_acc = 0

        count = 0
        for idx, batch in enumerate(train_loader): ### enum

            optim.zero_grad()
            batch_x = batch['data_x'].to(device)
            batch_y = batch['data_y'].to(device) ### .to(device) 추가
            ### to(device)를 안해줘서 cpu인지 gpu인지 모호해짐
            
            output = model.forward(batch_x) ### forward 추가

            loss = criterion(output, batch_y)
            avg_loss += loss.item()

            # cal accuracy
            _, index = torch.max(output, 1)
            avg_acc += (index == batch_y).sum().float() / len(batch_y)
            count += 1

            loss.backward()
            optim.step()

        avg_loss /= count
        avg_acc /= count

        # print("half success")

        # valid
        avg_test_loss = 0
        avg_test_acc = 0

        count = 0
        with torch.no_grad():    
            for idx, batch in enumerate(valid_loader): ### enum 수정
                # optim.zero_grad()
                batch_x = batch['data_x'].to(device) ### to.device 추가
                batch_y = batch['data_y'].to(device)
                output = model.forward(batch_x) ### forward 추가

                loss = criterion(output, batch_y)
                avg_test_loss += loss.item()

                # cal accuracy
                _, index = torch.max(output, 1)
                avg_test_acc += (index == batch_y).sum().float() / len(batch_y)
                count += 1

        avg_test_loss /= count
        avg_test_acc /= count

        print("[Epoch:%03d] train loss: %.5f train accuracy: %.4f | valid loss: %.5f valid accuracy: %.4f"
              % (ep+1, avg_loss, avg_acc, avg_test_loss, avg_test_acc))

    print("Training Done.")



if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Epoch:001] train loss: 0.90221 train accuracy: 0.6750 | valid loss: 0.59051 valid accuracy: 0.7737
[Epoch:002] train loss: 0.65934 train accuracy: 0.7584 | valid loss: 0.49380 valid accuracy: 0.8250
[Epoch:003] train loss: 0.59178 train accuracy: 0.7819 | valid loss: 0.46040 valid accuracy: 0.8336
[Epoch:004] train loss: 0.54460 train accuracy: 0.8015 | valid loss: 0.42770 valid accuracy: 0.8428
[Epoch:005] train loss: 0.52184 train accuracy: 0.8097 | valid loss: 0.42683 valid accuracy: 0.8464
[Epoch:006] train loss: 0.49800 train accuracy: 0.8192 | valid loss: 0.44268 valid accuracy: 0.8324
[Epoch:007] train loss: 0.48793 train accuracy: 0.8226 | valid loss: 0.39488 valid accuracy: 0.8549
[Epoch:008] train loss: 0.46822 train accuracy: 0.8292 | valid loss: 0.36329 valid accuracy: 0.8689
[Epoch:009] train loss: 0.46060 train accuracy: 0.8337 | valid loss: 0.